In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.util import dataframe_utils
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.objects.conversion.process_tree import converter as pt_converter
FILE = "BPI2016_Clicks_NOT_Logged_In.csv"


In [4]:
chunks = pd.read_csv(FILE, encoding='latin1',
    chunksize=5_000, delimiter=';')
df = next(chunks)

df.sort_values(['TIMESTAMP']).to_csv("temp.csv", index=False)

In [5]:
# total_df = pd.read_csv(FILE, encoding='latin1', delimiter=';')
# # set the page name to be null when the value is PORTAL.wwsbr_javascript.page_js
# total_df.loc[total_df['PAGE_NAME'] == 'PORTAL.wwsbr_javascript.page_js', 'PAGE_NAME'] = None
# total_df.dropna(subset=['PAGE_NAME'], inplace=True)
# # save the to the same file
# total_df.to_json('MODIFIED.json', index=False ,orient='records', lines=True)

In [6]:
# 1. Load CSV with the original delimiter and encoding
total_df = pd.read_csv(FILE, encoding='latin1', delimiter=';')

# 2. Remove rows with the specific unwanted value in PAGE_NAME
total_df = total_df[total_df['PAGE_NAME'] != 'PORTAL.wwsbr_javascript.page_js']
# filter out when page_action_detail set it to null when uwvsanlig-webfont.woff or if it contains woff
total_df.loc[total_df['page_action_detail'] == 'uwvsanlig-webfont.woff', 'page_action_detail'] = None
total_df = total_df[~total_df['page_action_detail'].str.contains('woff', na=False)]
# or remove all the PAGE_NAME that contain 'woff'
total_df = total_df[~total_df['PAGE_NAME'].str.contains('woff', na=False)]
# 3. Save back to the same file, keeping delimiter
total_df.to_csv(FILE, index=False, sep=';', encoding='latin1')


/var/folders/xr/8zbytzc93l76thl9y5ds98dr0000gn/T/ipykernel_2682/2803359812.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  total_df = pd.read_csv(FILE, encoding='latin1', delimiter=';')


In [7]:
# csv_row = []
# columns = []
# with open('BPI2016_Clicks_NOT_Logged_In.csv', 'r', encoding='utf-8') as f:
#     for i in range(30_000):
#         line = f.readline()
#         if i == 0:
#             columns = line.split(';')
#             continue
#         csv_row.append(line.split(';'))

# df = pd.DataFrame(csv_row, columns=columns)

In [8]:
# df = pd.read_csv('BPI2016_Clicks_NOT_Logged_In.csv', nrows=150, delimiter=';')
# df = next(chunks)

In [9]:
df.VHOST.value_counts()

VHOST
www.werk.nl      3623
digid.werk.nl    1377
Name: count, dtype: int64

In [10]:
# create here 
total_df.page_action_detail.value_counts()

page_action_detail
Zoek in werk.nl                                  767
beleidsregels                                    169
cv                                               161
afspiegeling                                     153
wajong                                           106
                                                ... 
plaatsingfee intermediar                           1
maximale leeftijd inschrijven UWV werkbedrijf      1
Werk                                               1
practicant                                         1
documenten toevoegen                               1
Name: count, Length: 6094, dtype: int64

In [11]:
# Step 1: Sort events by session and time
total_df = total_df.sort_values(by=["SessionID", "TIMESTAMP"])

# Step 2: Group by session and collect the ordered activity list as a tuple
traces = total_df.groupby("SessionID")["PAGE_NAME"].apply(tuple)

# Step 3: Count how many times each unique trace occurs
trace_counts = traces.value_counts()

# Step 4: Show top N traces
print(trace_counts.head(10))  # top 10 most common traces

# Total number of unique traces
print("Number of unique traces:", trace_counts.shape[0])

PAGE_NAME
(home,)                  905498
(werkmap,)               250525
(home, home)              73690
(timeout.htm,)            69392
(ww,)                     30501
(onderhoud.htm,)          29567
(vacatures,)              22474
(werkmap, home)           13191
(home, home, home)        12402
(bijstandsuitkering,)     12184
Name: count, dtype: int64
Number of unique traces: 197693


In [12]:
# Step 1: Sort events by session and time
total_df = total_df.sort_values(by=["SessionID", "TIMESTAMP"])

# Step 2: Group by session and collect the ordered activity list as a tuple
traces = total_df.dropna(subset=["page_action_detail"]).groupby("SessionID")["page_action_detail"].apply(tuple)

# Step 3: Count how many times each unique trace occurs
trace_counts = traces.value_counts()

# Step 4: Show top N traces
print(trace_counts.head(10))  # top 10 most common traces

# Total number of unique traces
print("Number of unique traces:", trace_counts.shape[0])

page_action_detail
(Zoek in werk.nl,)              480
(cv,)                            99
(beleidsregels,)                 95
(afspiegeling,)                  94
(werkplein,)                     73
(wajong,)                        67
(inschrijven,)                   55
(ioaw,)                          48
(test,)                          48
(vaststellingsovereenkomst,)     43
Name: count, dtype: int64
Number of unique traces: 5687


In [ ]:
print(trace_counts.head(12))  # top 10 most common traces


PAGE_NAME
(home,)                  905498
(werkmap,)               250525
(home, home)              73690
(timeout.htm,)            69392
(ww,)                     30501
(onderhoud.htm,)          29567
(vacatures,)              22474
(werkmap, home)           13191
(home, home, home)        12402
(bijstandsuitkering,)     12184
(timeout.htm, home)       10670
(Portal (fout),)           7712
Name: count, dtype: int64


In [ ]:
df.columns

Index(['SessionID', 'IPID', 'TIMESTAMP', 'VHOST', 'URL_FILE', 'PAGE_NAME',
       'REF_URL_category', 'page_load_error', 'page_action_detail', 'tip',
       'service_detail', 'xps_info', 'page_action_detail_EN',
       'service_detail_EN', 'tip_EN'],
      dtype='object')

In [13]:
total_df.service_detail.value_counts()

service_detail
2. Uw laatste werkgever        358989
4. Gewerkte uren               278448
6. Uw arbeidsverleden          152042
7. Toeslag                     143056
3. Het ontslag                 139273
1. Uw persoonlijke gegevens    125123
8. Versturen gegevens           89597
1. Uw situatie                  78942
5. Ander werk/inkomsten         68043
3. Uw bezittingen               51833
2. Uw inkomsten                 19701
6. Overige informatie           15801
7. Versturen gegevens           11067
4. Uw persoonlijke gegevens      9542
5. De arbeidsmarkt               7992
2. Toeslag                       2912
3. Versturen gegevens            1461
Name: count, dtype: int64

In [ ]:
total_df.SessionID.value_counts()[6000:6100]

SessionID
45798719    98
41148561    98
53090476    98
46499515    98
50212713    98
            ..
40564281    97
14596247    97
38515358    97
35253019    97
425783      97
Name: count, Length: 100, dtype: int64

In [15]:
total_df[total_df['SessionID'] == 157] # 

,SessionID,IPID,TIMESTAMP,VHOST,URL_FILE,PAGE_NAME,REF_URL_category,page_load_error,page_action_detail,tip,service_detail,xps_info,page_action_detail_EN,service_detail_EN,tip_EN
3511888,157,110048,2015-07-21 19:22:35.170000000,www.werk.nl,/werk_nl/werknemer/home,home,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3494204,157,110048,2015-07-21 19:23:01.157000000,www.werk.nl,/werk_nl/werknemer/home,home,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3496615,157,110048,2015-07-21 19:23:57.357000000,www.werk.nl,/werk_nl/werknemer/werkmap,werkmap,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3492998,157,110048,2015-07-21 19:26:45.650000000,www.werk.nl,/werk_nl/werknemer/home,home,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3516134,157,110048,2015-07-21 19:41:52.407000000,www.werk.nl,/werk_nl/werknemer/home,home,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3496333,157,110048,2015-07-21 19:42:36.627000000,www.werk.nl,/werk_nl/werknemer/werkmap,werkmap,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.xps_info.value_counts()

xps_info
image_DigiD                                                              2876
image_dienstverlening                                                     797
download_model_beeindigingsovereenkomst_dienstverband                     422
image_online trainingen                                                   289
download_model_arbeidsovereenkomst_voor_bepaalde_tijd                     247
                                                                         ... 
download_brochure_groothandel_samenvatting                                  1
image_173177                                                                1
image_212881                                                                1
image_1723035                                                               1
download_brochure_sectorbeschrijving_metalelektro_en_metaalnijverheid       1
Name: count, Length: 200, dtype: int64

In [36]:
# df.page_action_detail_EN.value_counts() # ,df.page_action_detail.value_counts() # .isnull().sum()
def column_details(series):
    print(f"Unique values: {series.value_counts()}")
    print(f"Missing values: {series.isnull().sum()}, total unique values : {series.nunique()}")
# df.page_action_detail_EN.isnull().sum() , df.page_action_detail.isnull().sum()

In [37]:
# df.to_csv('temp.csv',index=False)

In [38]:
# # remove all the rows where the REF_URL_category is less than 2% of the total rows
# percentage_threshold = 0.0002
# ref_url_counts = df.REF_URL_category.value_counts()
# ref_url_counts = ref_url_counts[ref_url_counts > percentage_threshold * len(df)]
# # df = df[df.REF_URL_category.isin(ref_url_counts.index)]
# df.REF_URL_category.value_counts()

In [39]:
# ref_url_counts.index
# df = df[df.REF_URL_category.isin(ref_url_counts.index)]

In [40]:
actions = df.page_action_detail.value_counts()
# save the actions to a csv file
actions.to_csv('actions.csv')

In [41]:
df.page_action_detail.value_counts()

page_action_detail
Zoek in werk.nl                                 82
cv                                              29
beleidsregels                                   16
afspiegeling                                    14
werkverkenner                                   12
                                                ..
formulieren                                      1
afspiegel                                        1
binnen welke tijd moet ww worden aangevraagd     1
apotheek                                         1
Administratief medewerker / servicedesk          1
Name: count, Length: 848, dtype: int64

In [42]:
df.tip.value_counts()


Series([], Name: count, dtype: int64)

In [43]:
service_detail_EN = df.service_detail_EN.value_counts()
# save the service_detail_EN to a csv file
service_detail_EN.to_csv('service_detail_EN.csv')

In [44]:
xps_info = df.xps_info.value_counts()
# save the xps_info to a csv file
xps_info.to_csv('xps_info.csv')

In [45]:
from collections import defaultdict

data_counts = defaultdict(lambda: defaultdict(int))

for chunk in chunks:  # assuming 'chunks' is your TextFileReader object
    for col in chunk.columns:
        value_counts = chunk[col].value_counts()
        for val, count in value_counts.items():
            data_counts[col][val] += count

# Example: print summary for one column
# print(data_counts['your_column_name'])

/var/folders/27/h86gj_fd3xg23kmxw_95c6kc0000gn/T/ipykernel_4896/947122960.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:  # assuming 'chunks' is your TextFileReader object
/var/folders/27/h86gj_fd3xg23kmxw_95c6kc0000gn/T/ipykernel_4896/947122960.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:  # assuming 'chunks' is your TextFileReader object
/var/folders/27/h86gj_fd3xg23kmxw_95c6kc0000gn/T/ipykernel_4896/947122960.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:  # assuming 'chunks' is your TextFileReader object
/var/folders/27/h86gj_fd3xg23kmxw_95c6kc0000gn/T/ipykernel_4896/947122960.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:  # assuming 'chunks' is your Te

In [46]:
data_counts['service_detail_EN']

defaultdict(int,
            {'2. Your last employer': 334891,
             '4. Hours worked': 260117,
             '6. Your employment history': 141603,
             '3. The dismissal': 130176,
             '7. Supplement': 133266,
             '1. Your personal details': 116409,
             '8. Send data': 83466,
             '1. Your situation': 72815,
             '5. Other work/income': 63372,
             '3. Your possessions': 48065,
             '2. Your income': 18222,
             '6. Other information': 14681,
             '7. Send data': 10288,
             '4. Your personal information': 8888,
             '5. The labor market': 7423,
             '2. Supplement': 2751,
             '3. Send data': 1376})

In [47]:
data_counts['service_detail']

defaultdict(int,
            {'2. Uw laatste werkgever': 334891,
             '4. Gewerkte uren': 260117,
             '6. Uw arbeidsverleden': 141603,
             '3. Het ontslag': 130176,
             '7. Toeslag': 133266,
             '1. Uw persoonlijke gegevens': 116409,
             '8. Versturen gegevens': 83466,
             '1. Uw situatie': 72815,
             '5. Ander werk/inkomsten': 63372,
             '3. Uw bezittingen': 48065,
             '2. Uw inkomsten': 18222,
             '6. Overige informatie': 14681,
             '7. Versturen gegevens': 10288,
             '4. Uw persoonlijke gegevens': 8888,
             '5. De arbeidsmarkt': 7423,
             '2. Toeslag': 2751,
             '3. Versturen gegevens': 1376})

In [48]:
sum(data_counts['REF_URL_category'].values()) * 0.02

20032.44

In [49]:
data_counts['REF_URL_category']

defaultdict(int,
            {'Logged Out': 338849,
             'Search Engine': 283696,
             'UWV': 253026,
             'DigID': 89368,
             'MyGovernment': 12079,
             'Logging In': 5838,
             'Mail Client': 7897,
             'Job Search Website': 5025,
             'FAQ': 2647,
             'Competence Atlas': 1569,
             'Analysis Tool': 1242,
             'Spider': 386})

In [50]:
# create a function that get the data_counts[column_name] and a threshold and return the values that are greater than the threshold
def filter_counts(data_counts , column_name, threshold):
    """Filter counts for a specific column based on a threshold."""
    total_counts = sum(data_counts[column_name].values())
    if total_counts == 0:
        print(f'This is 0 % of the total counts for {column_name}')
        return {}
    min_count = total_counts * threshold
    output = {k: v for k, v in data_counts[column_name].items() if v > min_count}
    total_counts_output = sum(output.values())
    print(f'This is {total_counts_output / total_counts * 100} % of the total counts for {column_name}')
    return output

In [51]:
filter_counts(data_counts , 'REF_URL_category', 0.02)

This is 96.3376403473566 % of the total counts for REF_URL_category


{'Logged Out': 338849, 'Search Engine': 283696, 'UWV': 253026, 'DigID': 89368}

In [52]:
df.columns

Index(['SessionID', 'IPID', 'TIMESTAMP', 'VHOST', 'URL_FILE', 'PAGE_NAME',
       'REF_URL_category', 'page_load_error', 'page_action_detail', 'tip',
       'service_detail', 'xps_info', 'page_action_detail_EN',
       'service_detail_EN', 'tip_EN'],
      dtype='object')

In [53]:
# Collect all relevant rows into a single DataFrame
event_logs = []

for chunk in chunks:
    # Keep only needed columns
    chunk = chunk[["SessionID", "PAGE_NAME", "TIMESTAMP"]].dropna()

    # Optional: remove sessions with only 1 event
    session_counts = chunk["SessionID"].value_counts()
    valid_sessions = session_counts[session_counts > 1].index
    chunk = chunk[chunk["SessionID"].isin(valid_sessions)]

    # Append cleaned chunk
    event_logs.append(chunk)

# Concatenate all
df_logs = pd.concat(event_logs)
df_logs = df_logs.sort_values(by=["SessionID", "TIMESTAMP"])  # ensure sorted

ValueError: No objects to concatenate

In [54]:
# Rename to match pm4py standards
df_logs.columns = ["case:concept:name", "concept:name", "time:timestamp"]

# Ensure correct types
df_logs = dataframe_utils.convert_timestamp_columns_in_df(df_logs)
# df_logs = dataframe_utils.ensure_datetime_format(df_logs)
df_logs["case:concept:name"] = df_logs["case:concept:name"].astype(str)

# Convert to event log
event_log = log_converter.apply(df_logs)

NameError: name 'df_logs' is not defined

In [ ]:
# Discover the process model
process_tree = inductive_miner.apply(event_log)

# Visualize the process tree
gviz = pt_visualizer.apply(process_tree)
pt_visualizer.view(gviz)

In [ ]:
# Convert tree to Petri net

net, initial_marking, final_marking = pt_converter.apply(process_tree)

from pm4py.visualization.petri_net import visualizer as pn_visualizer
gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)
